In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import time

In [2]:
pd.set_option('max_rows',2000)
pd.set_option('max_columns',2000)
np.set_printoptions()

In [3]:
#df = pd.read_stata('/scratch/sv1239/projects/mlcs/raw/Votelevel_stuffjan2013.dta')
df = pd.read_stata('Votelevel_stuffjan2013.dta')

In [4]:
df.shape

(56403, 868)

In [5]:
# columns to be removed, casenum or codej1... or j2vote... or j2maj... will be removed later
del_cols = ['fileid','cite','vol','beginpg','endopin','endpage','docnum','priorpub','_merge','year',
            'circuit','pseatno','decision_date','aatty_first_name','aatty_last_name','afirm_name',
            'ratty_first_name','ratty_last_name','rname_of_first_listed_amicus_gro','rfirm_namew','decisiondatenew2',
           'j1name','j2name','j3name','quartertoelect','pname','seatno','success','lsuc','ls1','ls2','ls3','lp',
            'lp2','lp3','sseatno','congress','congreso','afirst_listed_amicus_group','yearquarter','name','Name','State','j',
            'ids','codej4','j4vote1','j4vote2','j4maj1','j4maj2','codej5','j5vote1','j5vote2','j5maj1','j5maj2',
            'codej6','j6vote1','j6vote2','j6maj1','j6maj2','codej7','j7vote1','j7vote2','j7maj1','j7maj2',
            'codej8','j8vote1','j8vote2','j8maj1','j8maj2','codej9','j9vote1','j9vote2','j9maj1','j9maj2',
            'codej10','j10vote1','j10vote2','j10maj1','j10maj2','codej11','j11vote1','j11vote2','j11maj1','j11maj2',
            'codej12','j12vote1','j12vote2','j12maj1','j12maj2','codej13','j13vote1','j13vote2','j13maj1','j13maj2',
            'codej14','j14vote1','j14vote2','j14maj1','j14maj2','codej15','j15vote1','j15vote2','j15maj1','j15maj2','j16maj1','j16vote1']
#
# to_dummies = ['month','day','method','state','district','origin','source','distjudg',
#               'applfrom','adminrev','opinstat','treat','classact','crossapp','counsel1','counsel2','sanction',
#               'initiate','numappel','appnatpr','appnatpr','appbus','appnonp','appfed','appsubst','appstate',
#               'appfiduc','ap_stid','genapel1','bank_ap1','genapel2','bank_ap2','appel1','appel2',]

In [6]:
df.drop(labels=del_cols,axis=1,inplace=True)

In [7]:
df.shape

(56403, 761)

In [8]:
## I use case id to find the record of each vote of the same case, caseList save each unique case id for one time. 
## There are 18000+ unique cases.

caseList=[]
for i in range(df.shape[0]):
    if df.ix[i]["casenum"] not in caseList:
        caseList.append(df.ix[i]["casenum"])
print len(caseList)

18930


In [9]:
## Here every unique case correspond to one element(a sublist) in caserecord. 
## A sublist('combin') in caserecord has two dictionary, voterecord and majrecord, the record of each judges' votes and their alignment with majority. 
## I don't record the judges' roles in the original record(primary or second or third).
## The whole structure is like [ [{},{}], [{},{}],...], example: [[{123:1,135:1,555:3},{123:1,135:1,555:2}], [{...},{...}], ...]

## The newframe is sort of a replicate of the original dataframe, only that the order of its rows are better for later merge of sparse dataframe.
## For example, now the three vote records for case id 111 can be in index 1,125,823 of the original dataframe 'df', but in 'newframe', these record
## will in sequence and their index will match the other sparse dataframe.

caserecord=[]
newframe=pd.DataFrame()
for case in caseList:       ## I only ran caseList[:4], it costs about 1.5 seconds; for 19000 case it may take 1 hour in my computer.       
    combin=[]
    voterecord=dict()   ## to record the exact vote of each judge 
    majrecord=dict()   ## to record if the judges vote with majority
    subtest=df[df.casenum==case].reset_index(drop=True)  ## 'subtest' only take the records that have a specific case id
    num=subtest.shape[0]  ## usally num will be 3, because usally there are 3 records for each case 
    for j in range(num):
        
        if  (subtest.ix[j].codej2 not in voterecord) & pd.notnull(subtest.ix[j].codej2):  
            voterecord[subtest.ix[j].codej2]=subtest.ix[j].j2vote1
            majrecord[subtest.ix[j].codej2]=subtest.ix[j].j2maj1
            
        if  (subtest.ix[j].codej3 not in voterecord) & pd.notnull(subtest.ix[j].codej3):  
            voterecord[subtest.ix[j].codej3]=subtest.ix[j].j3vote1
            majrecord[subtest.ix[j].codej3]=subtest.ix[j].j3maj1
            
        if  (subtest.ix[j].codej1 not in voterecord) & pd.notnull(subtest.ix[j].codej1):  
            voterecord[subtest.ix[j].codej1]=subtest.ix[j].direct1
            majrecord[subtest.ix[j].codej1]=1
            
    assert len(voterecord)==len(majrecord)   ## make sure the two record, vote and alignment, match each other
    combin.append(voterecord)
    combin.append(majrecord)    
    caserecord.append(combin)
    for k in range(len(voterecord)):
        newframe=newframe.append(subtest.ix[0])
newframe=newframe.reset_index()              ## need to reset the index, otherwise will all be 0
newframe=newframe.drop('index',1)           ## drop the old index column 

In [49]:
## I put the the 2nd and 3rd judge in no difference, their attendences are presented in one dataframe, and vote result in another, instead of 
## making four dataframe in total.Because this is not recorded in the original data. For example, when we take the 3rd judge as primary, then 
## should we record the original primary judge as the 2nd judge or the 3rd judge? 
## It doesn't seem like a big difference between that we assign his/her role artificially 
## or not to assign the role and record 2nd and 3rd judge in the same dataframe. The latter way is easier to implement.

In [15]:
## this block will only return lists, I will turn lists into dataframe in the next block

attmatrix=[]    ## sparse dataframe that record the attendence of 2nd and 3rd judges
votematrix=[]   ## sparse dataframe that record the votes of 2nd and 3rd judges
majmatrix=[]    ## sparse dataframe that record the alignment of 2nd and 3rd judges
primatrix=[]    ## sparse dataframe that record the attendence of primary judge
output=[]       ## sparse dataframe that record the alignment of primary judge

for cell in caserecord:
    for k in cell[0]:
        celltem=deepcopy(cell[0])
        celltem.pop(k)
        votematrix.append(celltem)
        
        attmatrix.append(dict.fromkeys(celltem, 1))
        
        celltem=deepcopy(cell[1])
        
        temp=dict()
        temp[k]=1
        primatrix.append(temp)
        
        output.append(celltem[k])
        
        celltem.pop(k)
        majmatrix.append(celltem)
        

In [17]:
attmatrix=pd.DataFrame(attmatrix).fillna(value=0)
votematrix=pd.DataFrame(votematrix).fillna(value=0)
majmatrix=pd.DataFrame(majmatrix).fillna(value=0)
primatrix=pd.DataFrame(primatrix).fillna(value=0)
output=pd.DataFrame(output)
print attmatrix.shape,votematrix.shape,majmatrix.shape,primatrix.shape,output.shape

(12, 12) (12, 12) (12, 12) (12, 12) (12, 1)


In [41]:
## necessary for later merge, otherwise the columns name will have conflicts
attmatrix.columns = ['a'+str(x) for x in attmatrix.columns]
votematrix.columns = ['v'+str(x) for x in votematrix.columns]
majmatrix.columns = ['m'+str(x) for x in majmatrix.columns]
primatrix.columns = ['p'+str(x) for x in primatrix.columns]

In [19]:
## remove the columns that we are done with
newframe.drop(labels=['casenum','codej1','codej2','j2vote1','j2vote2','j2maj1',
                      'j2maj2','codej3','j3vote1','j3vote2','j3maj1','j3maj2'],axis=1,inplace=True)

In [21]:
new_cols = newframe .columns

In [22]:
new_cols=new_cols.tolist()

In [24]:
for col in keep_cols:
    if col in new_cols:
        new_cols.remove(col)

In [25]:
do_dummies=[]
for i in new_cols:
    if len(pd.unique(df[i]))>3:
        do_dummies.append(i)

In [31]:
newframe=pd.get_dummies(newframe,columns=do_dummies)

In [46]:
newframe=newframe.fillna(value=0)

In [42]:
MSD=attmatrix.join(votematrix.join(majmatrix.join(primatrix)))  ## MSD=Merge Sparse Dataframe

In [47]:
finalframe=newframe.join(MSD)

In [48]:
finalframe

,ElecYear_AndPrior,_Icircuit_1,_Icircuit_10,_Icircuit_11,_Icircuit_2,_Icircuit_3,_Icircuit_4,_Icircuit_5,_Icircuit_6,_Icircuit_7,_Icircuit_8,_Icircuit_9,_Igeniss2_1,_Igeniss2_2,_Igeniss2_3,_Igeniss2_6,_Igeniss2_7,_Ilastquart_1,_Iquarter_2,_Iquarter_3,_Iquarter_4,_Iquarterto_1,_Iquarterto_10,_Iquarterto_11,_Iquarterto_12,_Iquarterto_13,_Iquarterto_14,_Iquarterto_15,_Iquarterto_2,_Iquarterto_3,_Iquarterto_4,_Iquarterto_5,_Iquarterto_6,_Iquarterto_7,_Iquarterto_8,_Iquarterto_9,_Iyear_1926,_Iyear_1927,_Iyear_1928,_Iyear_1929,_Iyear_1930,_Iyear_1931,_Iyear_1932,_Iyear_1933,_Iyear_1934,_Iyear_1935,_Iyear_1936,_Iyear_1937,_Iyear_1938,_Iyear_1939,_Iyear_1940,_Iyear_1941,_Iyear_1942,_Iyear_1943,_Iyear_1944,_Iyear_1945,_Iyear_1946,_Iyear_1947,_Iyear_1948,_Iyear_1949,_Iyear_1950,_Iyear_1951,_Iyear_1952,_Iyear_1953,_Iyear_1954,_Iyear_1955,_Iyear_1956,_Iyear_1957,_Iyear_1958,_Iyear_1959,_Iyear_1960,_Iyear_1961,_Iyear_1962,_Iyear_1963,_Iyear_1964,_Iyear_1965,_Iyear_1966,_Iyear_1967,_Iyear_1968,_Iyear_1969,_Iyear_1970,_Iyear_1971,_Iyear_1972,_Iyear_1973,_Iyear_1974,_Iyear_1975,_Iyear_1976,_Iyear_1977,_Iyear_1978,_Iyear_1979,_Iyear_1980,_Iyear_1981,_Iyear_1982,_Iyear_1983,_Iyear_1984,_Iyear_1985,_Iyear_1986,_Iyear_1987,_Iyear_1988,_Iyear_1989,_Iyear_1990,_Iyear_1991,_Iyear_1992,_Iyear_1993,_Iyear_1994,_Iyear_1995,_Iyear_1996,_Iyear_1997,_Iyear_1998,_Iyear_1999,_Iyear_2000,_Iyear_2001,_Iyear_2002,affirmdummy,afghan,age50,appres,apptdemocrat,asame,asame_f1,asame_f2,asensame,asensame_f1,asensame_f2,boom,clinton,close,close2,close3,closerd,conc0pro0merdummy,conc0promerdummy,concmerdummy,concpro0merdummy,concpro2,concprodummy,concpromerdummy,concurauthor,concurdummy,concurjoin,count,crossa,d,d12,d13,d23,dAds3,dF1Ads3,dF2Ads3,dL1Ads3,dL2Ads3,dL3Ads3,dL4Ads3,dL5Ads3,dal,dar,decade2,democrat,dfl,dga,dhi,diss0pro0merdummy,diss0promerdummy,dissentdummy,dissentvote,dissmerdummy,disspro0merdummy,dissprodummy,disspromerdummy,districtjudgeinpanel,divided,divided1,dividedA,dks,dla,dropflag,dsc,dummy,dvt,electoralpct,electoralpivotal,elevated,elevated2,extra1,extra2,extra3,extra4,extreme,extremee,extremeleft,extremelefte,extremeright,extremerighte,extremescore,fartherd,female,fourties,fromdistrictcourt,gender,graddeg3,gulf,inexperience,initiate1,initiate2,initiate3,initiate4,initiate5,j1appres,j1distjudge,j1party,j1score,j2appres,j2distjudge,j2party,j2score,j3appres,j3distjudge,j3party,j3score,korean,last3,lastquarter,lastthreequarter,likely_elev2,logAds3,logF1Ads3,logF2Ads3,logL1Ads3,logL2Ads3,lq,lsuc2,m,majority,month911_3mon,month911_f1,month911_f2,month911_f3,month911_f4,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,moved,mrespty,nomoversinpanel,one,opp_wins,paag,pada,pag,pago,panelhas_s3judge,pausa,pbank,pcab,pcc,pccom,pccoun,pcomct,pcustom,pdat,pfedjdge,pgov,pgovt,phouse,pindreg1,pindreg2,pindreg3,pindreg4,pindreg5,plawprof,plocct,pmagis,pmayor,pname2,pname3,pname4,popularpct,pos11yrs,pos13,pos13yrs,pos14,pos14yrs,pos15,pos15yrs,pprivate,preg5,president,president_f1,president_f2,prespty,propneg,pscab,pseatno2,pseatno3,pseatno4,psenate,psg,psgo,pshouse,pslc,psp,pspros,pssc,pssenate,pterr,pusa,pusattorney,recent,recess,repres,repres_f1,repres_f2,retirenextyear,reversedummy,rsen_f1,rsen_f2,same,same_f1,same_f2,samescore,sanction,sayear,score,scoreI,scoreII,scoreIII,scoreIV,seatno3,seatno4,senate,senate_f1,senate_f2,sensame,sensame_f1,sensame_f2,signed,sittingoutside,sixmonthsafter,sixmonthsbefore,source1,source2,sseatno3,sseatno4,sseatno5,sseatno6,tamon,tayear,tids,type1,type2,type3,type4,type5,type6,type7,unan,unanimous,uporoutt,var10,var11,vietnam,war,ww2,Wlengthopin_2.0,Wlengthopin_5.0,Wlengthopin_7.0,Wlengthopin_11.0,Wopinionlenght_2.0,Wopinionlenght_5.0,Wopinionlenght_7.0,Wopinionlenght_11.0,Wtotalcites_1.0,Wtotalcites_3.0,Wtotalcites_5.0,Wtotalcites_28.0,___char_,abusedis_0.0,adminrev_15.0,age2trun_90.0,agen_acq_0.0,alj_0.0,altdisp_0.0,amicus_0.0,amicusapp_,amicusresp_,ap_stid_0.0,ap_stid_38.0,appbus_0.0,appel1_

In [ ]:
## the 'output' is the output